In [61]:
from multiprocessing.sharedctypes import Value
import numpy as np
import csv
import pandas as pd

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# data cleaning
def data_clean(data):
    # No use : 'PassengerId' 'Name' 'Ticket' 'Cabin'
	data = data.drop(['PassengerId'], axis=1)
	data = data.drop(['Name'], axis=1)
	data = data.drop(['Ticket'], axis=1)
	data = data.drop(['Cabin'], axis=1)

    # change sex to 0(female) or 1(male)
	data['Sex'] = pd.to_numeric(data['Sex'].str.replace("female", '0').replace("male", '1'))

    # Age 補 median
	data['Age'] = data['Age'].fillna(data['Age'].median())

    # Fare 補 mean
	data['Fare'] = data['Fare'].fillna(data['Fare'].mean())

    # Embarked: NaN S=1, C=2, Q=3
	data['Embarked'] = data['Embarked'].fillna("1")
	data['Embarked'] = pd.to_numeric(data['Embarked'].str.replace("S", '1').replace("C", '2').replace("Q", '3'))
	
	return data

Train = data_clean(train_data)
Test = data_clean(test_data)

In [62]:
print(Train)

     Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked
0           0       3    1  22.0      1      0   7.2500         1
1           1       1    0  38.0      1      0  71.2833         2
2           1       3    0  26.0      0      0   7.9250         1
3           1       1    0  35.0      1      0  53.1000         1
4           0       3    1  35.0      0      0   8.0500         1
..        ...     ...  ...   ...    ...    ...      ...       ...
886         0       2    1  27.0      0      0  13.0000         1
887         1       1    0  19.0      0      0  30.0000         1
888         0       3    0  28.0      1      2  23.4500         1
889         1       1    1  26.0      0      0  30.0000         2
890         0       3    1  32.0      0      0   7.7500         3

[891 rows x 8 columns]


In [63]:
print(Test)

     Pclass  Sex   Age  SibSp  Parch      Fare  Embarked
0         3    1  34.5      0      0    7.8292         3
1         3    0  47.0      1      0    7.0000         1
2         2    1  62.0      0      0    9.6875         3
3         3    1  27.0      0      0    8.6625         1
4         3    0  22.0      1      1   12.2875         1
..      ...  ...   ...    ...    ...       ...       ...
413       3    1  27.0      0      0    8.0500         1
414       1    0  39.0      0      0  108.9000         2
415       3    1  38.5      0      0    7.2500         1
416       3    1  27.0      0      0    8.0500         1
417       3    1  27.0      1      1   22.3583         2

[418 rows x 7 columns]


In [64]:
feature = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X_train = Train[feature]
Y_train = Train['Survived']

X_test = Test[feature]

In [65]:
print('Pearson correlation')
for i in feature:
    pccs = np.corrcoef(Train[i], Train['Survived'])
    print("{:<8}: {}".format(i, pccs[0,1]))

Pearson correlation
Pclass  : -0.3384810359610148
Sex     : -0.5433513806577546
Age     : -0.06491041993052583
SibSp   : -0.03532249888573557
Parch   : 0.08162940708348361
Fare    : 0.2573065223849622
Embarked: 0.10681138570891947


In [66]:
# RandomForest
from sklearn.ensemble import RandomForestClassifier
m1 = RandomForestClassifier(random_state = 1)

# train RandomForest model
m1.fit(X_train, Y_train)
m1.score(X_train, Y_train)

# prediction
RF_pred = m1.predict(X_test)
print(RF_pred)

[0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 1 0 1 0 0 1 1 1 1 1 0 1 1 1 0 0 0 1 0 1 1 1
 0 0 1 0 1 0 1 1 0 0 0 1 1 1 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 0 0
 1 0 0 1 0 1 1 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0
 1 1 0 1 0 0 1 1 1 1 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0
 1 0 1 0 0 1 0 0 1 0 1 1 1 1 1 1 0 0 0 0 1 0 0 0 0 0 0 1 1 1 1 1 0 1 1 0 1
 0 1 0 0 0 0 0 1 0 1 0 1 0 0 1 1 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 0 1 0 1 0
 1 1 1 1 0 1 0 0 0 1 0 0 1 0 0 1 1 1 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 1 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 1 0
 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 1 1 0 0 0 1 1
 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 0 0 1 0 0 1]


In [67]:
# Linear Regression
import numpy as np
from sklearn.linear_model import LinearRegression

# train LinearRegression model 
lm = LinearRegression()
lm.fit(X_train, Y_train)

# prediction
LR_pred = lm.predict(X_test)
LR_pred[LR_pred >= 0.5] = 1
LR_pred[LR_pred < 0.5] = 0
LR_pred = LR_pred.astype('int64')

print(LR_pred)

[0 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 0 1 0 0 0 0 0 1 1 0 0 1
 1 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 0
 1 1 0 1 0 1 0 0 0 0 0 0 1 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 0 0
 0 1 1 1 1 1 0 1 0 0 0]


In [68]:
# svm model
from sklearn.svm import SVC
svm = SVC(kernel = 'linear', probability = True)

# train svm model
svm.fit(X_train, Y_train)

# prediction
svm_pred = svm.predict(X_test)
print(svm_pred)

[0 1 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 0 0 1 0 1 0 1 0 0 0 0 0 1 1 0 0 1
 1 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 1 1 0 1 0
 1 0 0 1 0 1 0 0 0 0 0 0 1 1 1 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 1 0 1 0 0 0 0 1 1 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 1 0 1 1 0
 0 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 1 0 1 0 0 0]


In [69]:
# XGboost model
from xgboost import XGBClassifier

# train XGboost model
xg1 = XGBClassifier()
xg1 = xg1.fit(X_train, Y_train)

# prediction
XG_pred = xg1.predict(X_test)
print(XG_pred)

[0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 1 0 1 1 1 0 0 1 1 1 0 1 1 1 0 0 0 1 0 1 0 1
 0 0 1 0 1 0 1 1 0 0 1 1 1 0 0 1 1 0 0 0 0 0 1 0 1 0 1 1 1 1 0 0 1 1 0 0 0
 1 1 1 1 0 1 1 0 0 0 0 0 1 1 1 1 0 1 1 1 1 0 1 0 0 0 1 0 1 0 1 0 0 1 1 0 0
 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 0 0
 1 0 1 0 0 1 0 0 1 0 1 1 1 1 1 1 0 0 0 0 1 0 0 1 0 1 0 1 1 0 1 1 0 1 1 1 1
 0 1 0 0 0 0 0 0 0 1 0 1 0 0 1 1 1 0 1 0 1 0 0 1 0 0 0 0 1 0 0 1 0 1 1 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 1 0 0 1 1 1 0 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 1 0 1
 0 0 0 1 1 0 0 1 0 1 0 0 1 1 1 1 1 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 1 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 1 1 0 0 1 0 0 0 0 1 1 0 1 0 0 0 1 0 1
 1 0 0 1 0 0 1 0 0 0 1 0 1 0 0 0 1 1 0 0 0 0 0 1 0 1 1 0 1 1 1 1 0 0 0 1 1
 0 1 0 0 1 1 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 0 1 1 0 1 1]


In [70]:
predition_RF = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': RF_pred})
predition_RF.to_csv('submission_RF.csv', index = False)

In [71]:
predition_LR = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': LR_pred})
predition_LR.to_csv('submission_LR.csv', index = False)

In [72]:
predition_SVM = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': svm_pred})
predition_SVM.to_csv('submission_SVM.csv', index = False)

In [73]:
predition_XGboost = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': XG_pred})
predition_XGboost.to_csv('submission_XGboost.csv', index = False)

In [74]:
ensemble2_pred = (LR_pred + svm_pred)/2
ensemble2_pred[ensemble2_pred < 0.5] = 0
ensemble2_pred[ensemble2_pred >= 0.5] = 1
ensemble2_pred = ensemble2_pred.astype("int64")
print(ensemble2_pred)

predition_ensemble2 = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': ensemble2_pred})
predition_ensemble2.to_csv('submission_ensemble2.csv', index = False)

[0 1 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 0 1 0 0 0 0 0 1 1 0 0 1
 1 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 0
 1 1 0 1 0 1 0 0 0 0 0 0 1 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 1 0 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1 1 0 1 0 0 0 0 1 1 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 1 0 1 1 0
 0 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 0 0
 0 1 1 1 1 1 0 1 0 0 0]


In [75]:
ensemble3_pred = (RF_pred + LR_pred + svm_pred)/3
ensemble3_pred[ensemble3_pred < 0.5] = 0
ensemble3_pred[ensemble3_pred >= 0.5] = 1
ensemble3_pred = ensemble3_pred.astype("int64")
print(ensemble3_pred)

predition_ensemble3 = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': ensemble3_pred})
predition_ensemble3.to_csv('submission_ensemble3.csv', index = False)

[0 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 0 1 0 0 0 0 0 1 1 0 0 1
 1 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 0
 1 0 0 1 0 1 0 0 0 0 0 0 1 1 1 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0 0 1 0 1 0 0 0 0 1 1 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 1 0 1 0 0 0]


In [76]:
ensemble_pred = (RF_pred + LR_pred + svm_pred + XG_pred)/4
ensemble_pred[ensemble_pred < 0.5] = 0
ensemble_pred[ensemble_pred >= 0.5] = 1
ensemble_pred = ensemble_pred.astype("int64")
print(ensemble_pred)

predition_ensemble = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': ensemble_pred})
predition_ensemble.to_csv('submission_ensemble.csv', index = False)

[0 0 0 1 1 0 1 0 1 0 0 0 1 0 1 1 0 1 1 1 0 0 1 1 1 0 1 1 1 0 0 0 1 1 1 0 1
 1 0 1 0 1 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 0
 1 1 0 1 0 1 1 0 0 0 0 0 1 1 1 1 1 0 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 1 0 0
 1 1 1 1 0 0 1 1 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0
 1 0 1 0 0 1 0 0 1 1 1 1 1 1 1 1 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 1 1 0 1
 0 1 0 0 0 0 0 0 0 1 0 1 1 0 1 1 1 0 1 0 1 1 0 1 0 0 0 0 1 1 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 1 0 0 1 1 1 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0
 1 0 0 1 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 1 1 0 0 1 1 1
 0 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 1 0 1 0 0 1]
